[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/langchain-ai/langchain/blob/master/docs/docs/use_cases/tagging.ipynb)

## Use case | ユースケース

タグ付けとは、以下のようなクラスを用いてドキュメントにラベルを付けることを意味します：

> Tagging means labeling a document with classes such as:

* 感情
  > sentiment
* 言語
  > language
* スタイル（フォーマル、インフォーマルなど）
  > style (formal, informal etc.)
* 取り上げたトピック
  > covered topics
* 政治的傾向
  > political tendency

![Image description](../../static/img/tagging.png)

## Overview | 概要

タグ付けにはいくつかの要素があります：

> Tagging has a few components:

* `function`：[抽出](/docs/use_cases/extraction)と同様に、タギングにおいてもモデルがドキュメントにタグを付ける方法を指定するために[関数](https://openai.com/blog/function-calling-and-other-api-updates)を使用します
  > `function`: Like [extraction](/docs/use_cases/extraction), tagging uses [functions](https://openai.com/blog/function-calling-and-other-api-updates) to specify how the model should tag a document
* `schema`は、ドキュメントにタグを付ける際の形式を定義します
  > `schema`: defines how we want to tag the document

## Quickstart | クイックスタート

LangChainでOpenAIの機能をタグ付けにどのように使用できるか、非常にわかりやすい例を見てみましょう。

> Let's see a very straightforward example of how we can use OpenAI functions for tagging in LangChain.


In [ ]:
!pip install langchain openai

# Set env var OPENAI_API_KEY or load from a .env file:
# import dotenv
# dotenv.load_dotenv()

In [1]:
from langchain.chains import create_tagging_chain, create_tagging_chain_pydantic
from langchain.chat_models import ChatOpenAI

私たちは、スキーマ内でいくつかのプロパティとそれらの期待される型を指定します。

> We specify a few properties with their expected type in our schema.


In [2]:
# Schema
schema = {
    "properties": {
        "sentiment": {"type": "string"},
        "aggressiveness": {"type": "integer"},
        "language": {"type": "string"},
    }
}

# LLM
llm = ChatOpenAI(temperature=0, model="gpt-3.5-turbo-0613")
chain = create_tagging_chain(schema, llm)

In [4]:
inp = "Estoy increiblemente contento de haberte conocido! Creo que seremos muy buenos amigos!"
chain.run(inp)

{'sentiment': 'positive', 'language': 'Spanish'}

In [5]:
inp = "Estoy muy enojado con vos! Te voy a dar tu merecido!"
chain.run(inp)

{'sentiment': 'enojado', 'aggressiveness': 1, 'language': 'es'}

例で見ることができる通り、それは私たちが望むことを正確に解釈しています。

> As we can see in the examples, it correctly interprets what we want.

結果は変動し、例えば異なる言語での感情（'positive', 'enojado' など）が得られます。

> The results vary so that we get, for example, sentiments in different languages ('positive', 'enojado' etc.).

次のセクションで、これらの結果をどのように制御するかを見ていきます。

> We will see how to control these results in the next section.


## Finer control | より細かい制御

慎重なスキーマ定義によって、モデルの出力をよりコントロールすることができます。

> Careful schema definition gives us more control over the model's output.

具体的には、以下のように定義できます：

> Specifically, we can define:

* 各プロパティの可能な値
  > possible values for each property
* モデルがプロパティを理解していることを確認するための説明
  > description to make sure that the model understands the property
* 返されるべき必須プロパティ
  > required properties to be returned


以下に、`_enum_`、`_description_`、そして`_required_`を使用して、先に述べた各側面を制御する例を示します：

> Here is an example of how we can use `_enum_`, `_description_`, and `_required_` to control for each of the previously mentioned aspects:


In [6]:
schema = {
    "properties": {
        "aggressiveness": {
            "type": "integer",
            "enum": [1, 2, 3, 4, 5],
            "description": "describes how aggressive the statement is, the higher the number the more aggressive",
        },
        "language": {
            "type": "string",
            "enum": ["spanish", "english", "french", "german", "italian"],
        },
    },
    "required": ["language", "sentiment", "aggressiveness"],
}

In [7]:
chain = create_tagging_chain(schema, llm)

これで回答がずっと良くなりました！

> Now the answers are much better!


In [10]:
inp = "Estoy increiblemente contento de haberte conocido! Creo que seremos muy buenos amigos!"
chain.run(inp)

{'aggressiveness': 0, 'language': 'spanish'}

In [11]:
inp = "Estoy muy enojado con vos! Te voy a dar tu merecido!"
chain.run(inp)

{'aggressiveness': 5, 'language': 'spanish'}

In [12]:
inp = "Weather is ok here, I can go outside without much more than a coat"
chain.run(inp)

{'aggressiveness': 0, 'language': 'english'}

[LangSmith trace](https://smith.langchain.com/public/311e663a-bbe8-4053-843e-5735055c032d/r)を使うと、内部の動作を覗き見ることができます。

> The [LangSmith trace](https://smith.langchain.com/public/311e663a-bbe8-4053-843e-5735055c032d/r) lets us peek under the hood:

* [抽出](/docs/use_cases/extraction)と同様に、入力文字列に対して`information_extraction`関数を[こちら](https://github.com/langchain-ai/langchain/blob/269f85b7b7ffd74b38cd422d4164fc033388c3d0/libs/langchain/langchain/chains/openai_functions/extraction.py#L20)で呼び出します。
  > As with [extraction](/docs/use_cases/extraction), we call the `information_extraction` function [here](https://github.com/langchain-ai/langchain/blob/269f85b7b7ffd74b38cd422d4164fc033388c3d0/libs/langchain/langchain/chains/openai_functions/extraction.py#L20) on the input string.
* このOpenAIの機能は、提供されたスキーマに基づいて情報を抽出します。
  > This OpenAI function extraction information based upon the provided schema.

![Image description](../../static/img/tagging_trace.png)


## Pydantic | Pydantic


また、必要なプロパティと型を指定するために、Pydanticスキーマを使用することもできます。

> We can also use a Pydantic schema to specify the required properties and types.

また、`enum`や`description`のような他の引数を各フィールドに送信することもできます。

> We can also send other arguments, such as `enum` or `description`, to each field.

これにより、私たちは純粋なPythonの型を用いて、Pythonで新しいクラスや関数を定義するのと同じ方法でスキーマを指定することができます。

> This lets us specify our schema in the same manner that we would a new class or function in Python with purely Pythonic types.


In [5]:
from langchain_core.pydantic_v1 import BaseModel, Field

In [7]:
class Tags(BaseModel):
    sentiment: str = Field(..., enum=["happy", "neutral", "sad"])
    aggressiveness: int = Field(
        ...,
        description="describes how aggressive the statement is, the higher the number the more aggressive",
        enum=[1, 2, 3, 4, 5],
    )
    language: str = Field(
        ..., enum=["spanish", "english", "french", "german", "italian"]
    )

In [8]:
chain = create_tagging_chain_pydantic(Tags, llm)

In [9]:
inp = "Estoy muy enojado con vos! Te voy a dar tu merecido!"
res = chain.run(inp)

In [10]:
res

Tags(sentiment='sad', aggressiveness=5, language='spanish')

### Going deeper | さらに深く

* [metadata tagger](https://python.langchain.com/docs/integrations/document_transformers/openai_metadata_tagger)ドキュメントトランスフォーマーを使用して、LangChainの`Document`からメタデータを抽出することができます。
  > You can use the [metadata tagger](https://python.langchain.com/docs/integrations/document_transformers/openai_metadata_tagger) document transformer to extract metadata from a LangChain `Document`.
* これはタギングチェーンと同じ基本的な機能を提供しますが、LangChainの`Document`にのみ適用されるという点が異なります。
  > This covers the same basic functionality as the tagging chain, only applied to a LangChain `Document`.
